In [2]:
# Import necessary packages
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.estimators import BayesianEstimator
import pyro
import pyro.distributions as dist
import torch

In [3]:
# Read in the dataset
data = pd.read_csv("./dag_data.csv")

# Construct a DAG model in Pyro to experiment with interventions and conditional modeling

### 1. Construct a DAG using the pgmpy Python package, fit the DAG on the dataset, then   extract conditional probability tables for each variable.

#### Construct a DAG using the pgmpy package.
We will model interventions and inferences on the DAG using pyro, but loading the data into a pygmy Bayesian network will allow us to easily transfer large conditional probability tables to pyro. 

In [4]:
# Create a Bayesian model using pgmpy to represent the DAG.
bayes_model = BayesianModel([('CPL', 'CDC'), ('HS', 'CDC'), ('MI', 'CDC'), ('CDC', 'CC'), 
                          ('UI', 'ICU'),('ICU', 'CC'), ('SPL', 'BLG'), ('BLG', 'CC')])

#### Fit the model on the cleaned dataset.

In [5]:
# Fit the Bayesian model on the dataset.
bayes_model.fit(data)

#### Extract conditional probability tables for every variable in the DAG.

In [7]:
# Extract conditional probability tables from each variable in the DAG.
cpl_probs = torch.tensor(bayes_model.get_cpds(node="CPL").values)
cdc_probs = torch.tensor(bayes_model.get_cpds(node="CDC").values)
hs_probs = torch.tensor(bayes_model.get_cpds(node="HS").values)
mi_probs = torch.tensor(bayes_model.get_cpds(node="MI").values)
ui_probs = torch.tensor(bayes_model.get_cpds(node="UI").values)
icu_probs = torch.tensor(bayes_model.get_cpds(node="ICU").values)
spl_probs = torch.tensor(bayes_model.get_cpds(node="SPL").values)
blg_probs = torch.tensor(bayes_model.get_cpds(node="BLG").values)
cc_probs = torch.tensor(bayes_model.get_cpds(node="CC").values)

### 2. Construct a DAG in Pyro, built upon the conditional probability tables extracted from the pgmpy Bayesian network.

In [8]:
def cc_pyro_model():
    CPL = pyro.sample("CPL", dist.Categorical(probs=cpl_probs)) 
    HS = pyro.sample("HS", dist.Categorical(probs=hs_probs)) 
    MI = pyro.sample("MI", dist.Categorical(probs=mi_probs)) 
    UI = pyro.sample("UI", dist.Categorical(probs=ui_probs))
    SPL = pyro.sample("SPL", dist.Categorical(probs=spl_probs))
    CDC = pyro.sample("CDC", dist.Categorical(probs=cdc_probs[CPL][HS][MI]))  # ???????
    ICU = pyro.sample("ICU", dist.Categorical(probs=icu_probs[UI]))
    BLG = pyro.sample("BLG", dist.Categorical(probs=blg_probs[SPL]))
    CC = pyro.sample("CC", dist.Categorical(probs=cc_probs[CDC][ICU][BLG])) # ????????
    return{"CPL": CPL, "CDC": CDC, "HS" : HS, "MI": MI, "UI": UI,
           "ICU": ICU, "SPL" : SPL, "BLG" : BLG, "CC" : CC} 

print(cc_pyro_model())

{'CPL': tensor(3), 'CDC': tensor(2), 'HS': tensor(1), 'MI': tensor(3), 'UI': tensor(2), 'ICU': tensor(1), 'SPL': tensor(1), 'BLG': tensor(0), 'CC': tensor(3)}


##  ^ TODO: Look at the order of conditional variables in the above CDC and CC conditional probability tensors. How should they be ordered? I guessed on the order of tensors

# Test Hypotheses about the Data

### Hypothesis 1. Counties in which there are fewer ICU beds report fewer COVID-19 confirmed cases per 100,000 people. 

In [15]:
# Get the tensor index that corresponds to ICU = low
bayes_model.get_cpds(node="ICU").state_names["ICU"]


['High', 'Low', 'Medium', 'Very High']

In [16]:
# Create conditioned model where ICU = "low"
icu_conditioned_model = pyro.condition(cc_pyro_model, data={'ICU':torch.tensor(1)})

## TODO: fix the below code

In [14]:
# Gather 1000 sample values of T
T_samples = [icu_conditioned_model()['CC'] for _ in range(1000)] 
T_unique, T_counts = np.unique(T_samples, return_counts=True)

RuntimeError: invalid multinomial distribution (encountering probability entry < 0)

In [ ]:

# Plot outcome values of T
plt.bar(T_unique, T_counts, align='center', alpha=0.5)
plt.xticks(T_unique, T_alias)
plt.ylabel('count')
plt.xlabel('County Number Confirmed Cases')
plt.title('Number of Confirmed Cases Among Counties when ICU bed count is Low')

###  Hypothesis 2. Counties that ban large gatherings report more COVID-19 confirmed cases per 100,000 people.